# Simple Graph-Synchronization Stitching

This notebook is used to perform stitching exploiting the graph synchronization method.

## Importing libraries

In [1]:
import cv2 as cv
import os
import shutil
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import ipynb.fs.defs.Utils as Utils
import ipynb.fs.defs.ImageStitcher as ImageStitcher

## Functions definition

In [2]:
#This function allows to perform simple graph-synchronization stitching
def simple_graph_stitching(dataset_name, #Name of the dataset to be used
                            imgs, #Images to be stitched
                            T_norm, #Normalization matrix
                            M, #Matrix containing the homographies between the images (TODO)
                            idx_ref = 0, #Index of the reference image
                            idxs = None, #Indexes of the subset of images to be stitched
                            verbose = True, #If True allows to print intermediate results
                            save_output = True, #If True allows to save the output
                            beautify = True, #If True allows to print the stitched image in a better way
                            stitching_dir = "stitched", #Directory where to save the results
                            graph_stitching_dir = "simple_graph_stitching", #Directory where to save the obtained graph
                            warp_shape = [10000,10000] #Shape of the stitched image
                          ):
    
    #Output directory
    output_dir = os.path.join(os.path.join(stitching_dir,dataset_name),graph_stitching_dir)
    #Check whether the directory exists and create it
    if save_output:
        if os.path.isdir(output_dir):   
            shutil.rmtree(output_dir)
        if not os.path.isdir(output_dir):   
            os.makedirs(output_dir)
            
    #u: left eigenvectors
    #d: eigenvalues
    #vh: right eigenvectors
    u,d,vh = np.linalg.svd(M) #Application of SVD to the M matrix
    v = vh.transpose() #Transpose matrix of right eigenvectors
    
    # Find the three right singular vectors associated with the 3 smallest singular values (so the last 3 columns)
    u_hat = v[:,[-1,-2,-3]]
    
    #Get the state of each node
    U = Utils.split_states(u_hat)
    
    #Retrieve the homographies w.r.t. the reference image
    H = Utils.get_homographies_from_states(U, idx_ref)
    
    #Stitch images together
    stitcher = ImageStitcher.ImageStitcher(imgs)
    H, stitch = stitcher.stitch_images(H, idx_ref, T_norm, idxs , beautify=beautify, size = warp_shape) #Stitch all the images
    
    #Save and print output if required
    if save_output:
        cv.imwrite(os.path.join(output_dir,"stitched.jpg"), cv.cvtColor(stitch,cv.COLOR_RGB2BGR))
    
    if verbose:
        figure(figsize=(40, 40), dpi=80)
        plt.imshow(stitch,),plt.show() 
    
    return H, stitch